# Population Density of Mexican States and Municipalities

This Notebook downloads Geopandas GeoDataFrames for States (admin1) and Municipalities (admin2) derived from the 2020 Mexican Census: [INEGI](https://www.inegi.org.mx/temas/mg/).

For details how these dataframe were created, see the [mexican-boundaries](https://github.com/sbl-sdsc/mexico-boundaries) GitHub project.

In [1]:
from io import BytesIO
from urllib.request import urlopen
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

## Boundaries of Mexican States

Read boundary polygons for Mexican states from shapefile

In [3]:
admin1_url = 'https://raw.githubusercontent.com/sbl-sdsc/mexico-boundaries/main/data/mexico_admin1.parquet'

In [4]:
resp = urlopen(admin1_url)
admin1 = gpd.read_parquet(BytesIO(resp.read()))

Calculate the area of each state (convert area from m^2 to km^2

In [5]:
admin1.crs

<Projected CRS: PROJCRS["MEXICO_ITRF_2008_LCC",BASEGEOGCRS["MEXICO ...>
Name: MEXICO_ITRF_2008_LCC
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: International Terrestrial Reference Frame 2008
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
admin1['area_sqkm'] = admin1['geometry'].area/ 10**6

In [7]:
admin1.head()

,CVEGEO,CVE_ENT,NOMGEO,geometry,area_sqkm
0,01,01,Aguascalientes,"POLYGON ((2470517.824 1155028.588, 2470552.248...",5558.673843
1,02,02,Baja California,"MULTIPOLYGON (((1493197.166 1849625.247, 14934...",73516.697324
2,03,03,Baja California Sur,"MULTIPOLYGON (((1694656.344 1227647.637, 16946...",73970.687326
3,04,04,Campeche,"MULTIPOLYGON (((3544897.199 946994.621, 354491...",57254.511636
4,05,05,Coahuila de Zaragoza,"POLYGON ((2469954.193 1978522.993, 2469982.807...",150671.222986


Get population from week 1 analyzes data file

In [8]:
population_admin1 = pd.read_csv('../data/week1analyzesStates.csv', 
                                usecols=['cve_ent', 'population'])

In [9]:
population_admin1.head()

,cve_ent,population
0,1,1434635
1,2,3634868
2,3,804708
3,4,1000617
4,7,5730367


Add CVE_ENT state code column (example: convert 1 -> 01)

In [10]:
population_admin1['CVE_ENT'] = population_admin1['cve_ent'].apply(lambda i: f'{i:02d}')

In [11]:
population_admin1.head()

,cve_ent,population,CVE_ENT
0,1,1434635,01
1,2,3634868,02
2,3,804708,03
3,4,1000617,04
4,7,5730367,07


Merge the geo dataframe with the population dataframe using the common CVE_ENT column

In [12]:
df_admin1 = admin1.merge(population_admin1, on='CVE_ENT')

In [13]:
df_admin1.head()

,CVEGEO,CVE_ENT,NOMGEO,geometry,area_sqkm,cve_ent,population
0,01,01,Aguascalientes,"POLYGON ((2470517.824 1155028.588, 2470552.248...",5558.673843,1,1434635
1,02,02,Baja California,"MULTIPOLYGON (((1493197.166 1849625.247, 14934...",73516.697324,2,3634868
2,03,03,Baja California Sur,"MULTIPOLYGON (((1694656.344 1227647.637, 16946...",73970.687326,3,804708
3,04,04,Campeche,"MULTIPOLYGON (((3544897.199 946994.621, 354491...",57254.511636,4,1000617
4,05,05,Coahuila de Zaragoza,"POLYGON ((2469954.193 1978522.993, 2469982.807...",150671.222986,5,3218720


The population density is computed

In [14]:
df_admin1['population/sqkm']=df_admin1['population']/df_admin1['area_sqkm']

In [15]:
df_admin1.head()

,CVEGEO,CVE_ENT,NOMGEO,geometry,area_sqkm,cve_ent,population,population/sqkm
0,01,01,Aguascalientes,"POLYGON ((2470517.824 1155028.588, 2470552.248...",5558.673843,1,1434635,258.089437
1,02,02,Baja California,"MULTIPOLYGON (((1493197.166 1849625.247, 14934...",73516.697324,2,3634868,49.442754
2,03,03,Baja California Sur,"MULTIPOLYGON (((1694656.344 1227647.637, 16946...",73970.687326,3,804708,10.878742
3,04,04,Campeche,"MULTIPOLYGON (((3544897.199 946994.621, 354491...",57254.511636,4,1000617,17.476649
4,05,05,Coahuila de Zaragoza,"POLYGON ((2469954.193 1978522.993, 2469982.807...",150671.222986,5,3218720,21.362540


The data columns of interest is selected for future storage

In [16]:
dfFinal = df_admin1[['cve_ent','population/sqkm']].copy()
dfFinal.head()

,cve_ent,population/sqkm
0,1,258.089437
1,2,49.442754
2,3,10.878742
3,4,17.476649
4,5,21.362540


The dataframe is stored

In [17]:
dfFinal.to_csv('../data/week3_population_density_states.csv')